In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fitness-tracker-data/Fitness_Tracker_Data.csv


# DATA LOADING AND PREPROCESSING

In [2]:
df = pd.read_csv('/kaggle/input/fitness-tracker-data/Fitness_Tracker_Data.csv')
df

,User_ID,Date,Steps,Heart_Rate_avg,Calories_Burned,Workout_Type
0,4001,2024-01-01,14748,68,598,Cardio
1,4002,2024-01-02,8943,74,428,Cardio
2,4003,2024-01-03,14195,69,724,NaN
3,4004,2024-01-04,7963,97,284,Strength
4,4005,2024-01-05,14671,93,622,Yoga
...,...,...,...,...,...,...
95,4096,2024-04-05,7333,76,299,Yoga
96,4097,2024-04-06,11548,63,393,Yoga
97,4098,2024-04-07,12708,85,443,Cardio
98,4099,2024-04-08,6781,73,711,Yoga


In [3]:
df.isna().sum()

User_ID             0
Date                0
Steps               0
Heart_Rate_avg      0
Calories_Burned     0
Workout_Type       21
dtype: int64

# REPLACING NaN VALUES

In [4]:
def fill_missing_values(df):
    df_filled = df.copy()

    for col in df.columns:
        if df[col].dtype == 'object':
            non_null_values = df[col].dropna().unique()
            num_missing = df[col].isna().sum()
        
        
            if num_missing > 0 and len(non_null_values) > 0:
                replacements = np.tile(non_null_values,int(np.ceil(num_missing / len(non_null_values))))
                np.random.shuffle(replacements)
                df_filled.loc[df[col].isna(),col] = replacements[:num_missing]

    return df_filled

In [5]:
df_filled = fill_missing_values(df)
df_filled

,User_ID,Date,Steps,Heart_Rate_avg,Calories_Burned,Workout_Type
0,4001,2024-01-01,14748,68,598,Cardio
1,4002,2024-01-02,8943,74,428,Cardio
2,4003,2024-01-03,14195,69,724,Cardio
3,4004,2024-01-04,7963,97,284,Strength
4,4005,2024-01-05,14671,93,622,Yoga
...,...,...,...,...,...,...
95,4096,2024-04-05,7333,76,299,Yoga
96,4097,2024-04-06,11548,63,393,Yoga
97,4098,2024-04-07,12708,85,443,Cardio
98,4099,2024-04-08,6781,73,711,Yoga


# FEATURE ENGINEERING

In [6]:
from sklearn.preprocessing import LabelEncoder

In [7]:
le = LabelEncoder()

In [8]:
df['Workout_Type'] = le.fit_transform(df['Workout_Type'])

In [9]:
df

,User_ID,Date,Steps,Heart_Rate_avg,Calories_Burned,Workout_Type
0,4001,2024-01-01,14748,68,598,0
1,4002,2024-01-02,8943,74,428,0
2,4003,2024-01-03,14195,69,724,3
3,4004,2024-01-04,7963,97,284,1
4,4005,2024-01-05,14671,93,622,2
...,...,...,...,...,...,...
95,4096,2024-04-05,7333,76,299,2
96,4097,2024-04-06,11548,63,393,2
97,4098,2024-04-07,12708,85,443,0
98,4099,2024-04-08,6781,73,711,2


# ANN MODEL

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam

2025-06-02 14:51:59.982926: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748875920.290914      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748875920.374679      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,r2_score,accuracy_score,recall_score,classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dropout, BatchNormalization

In [12]:
X = df.drop(columns=['Workout_Type','User_ID','Date'])
y = df['Workout_Type']

In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [14]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [15]:
num_classes = 4  # Change this based on your dataset
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

2025-06-02 14:52:15.851390: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [16]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # use categorical_crossentropy if labels are one-hot encoded
              metrics=['accuracy'])


In [17]:
# model.fit(X_train_scaled, y_train, epochs=10, batch_size=32)
from sklearn.utils import class_weight

classes = np.unique(y_train)
class_weights_array = class_weight.compute_class_weight('balanced', classes=classes, y=y_train)

# Convert array to dict: {class_label: weight}
class_weights = dict(zip(classes, class_weights_array))

# Now use in model.fit()
model.fit(X_train, y_train, epochs=50, batch_size=32, class_weight=class_weights)


Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.2227 - loss: 1.8463
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2367 - loss: 1.7186
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2383 - loss: 1.5984
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3523 - loss: 1.5422
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.2328 - loss: 1.5135
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2133 - loss: 1.6340
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3016 - loss: 1.4254
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3328 - loss: 1.5975
Epoch 9/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.1547 - loss: 1.5708
Epoch 10/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2672 - loss: 1.4828
Epoch 11/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2219 - loss: 1.4662
Epoch 12/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3305 - loss: 1.5069
E

In [18]:
# Predict probabilities for each class
y_probs = model.predict(X_test_scaled)

# Get class with highest probability
y_pred = y_probs.argmax(axis=1)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step


In [19]:
print(y_pred)


[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]


In [20]:
# Print accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

# Print detailed classification report (precision, recall, f1-score)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Print confusion matrix
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.15

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.00      0.00      0.00         6
           2       0.00      0.00      0.00         8
           3       0.15      1.00      0.26         3

    accuracy                           0.15        20
   macro avg       0.04      0.25      0.07        20
weighted avg       0.02      0.15      0.04        20


Confusion Matrix:
 [[0 0 0 3]
 [0 0 0 6]
 [0 0 0 8]
 [0 0 0 3]]


# CONCLUSION:

> NEED TO MORE WORK ON IT..
>
> MAY BE BY USING DATA-AUGMENTATION WE CAN IMPORVE THE RESULTS.